In [1]:
#!pip install eunjeon #은전 프로젝트 패키지, Mecab 사용 위함.

#밑에 없는 패키지들은
#!pip install 패키지명
#으로 설치하면 됩니다.

In [8]:
import os, re, csv
import pandas as pd
from eunjeon import Mecab
from tqdm import tqdm
import gensim
#import soynlp
mecab = Mecab()

eta_df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\질문2.csv", encoding='ANSI')

eta_df.shape

(3024, 6)

# dataframe에서 댓글, 내용 리스트로 가져오기

In [9]:
comments = eta_df['댓글'].tolist()

In [10]:
content = eta_df['내용'].tolist()

# 전처리 (정규화, 불용어 제거)

In [12]:
file = open(r'E:\Programming\python\NLP\howls_nlp\Chatbot\자료\stop_words.txt','r', encoding = 'utf8')
lines = file.readlines()
file.close()
stop_words = lines[0].split()

#print(stop_words)

In [13]:
norm_content = []
def text_normalization(content):
    result = []
    for text in content:
        punct = re.compile(r'[\~\'\"\.\,\?\!\:\;\)\(\+]+') 
        norm_text = re.sub(punct, "", text)
        norm_text = re.sub(r'안녕(\s)*(하세요)?', '', norm_text) #안녕, 안녕하세요, 안녕 하세요...
        norm_text = re.sub(r'(조교)[님]?', "", norm_text) #조교, 조교님
        norm_text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]{2,}', ' ', norm_text) #2개 이상 자모 연속 제거 
        norm_text = re.sub( r"\s{2,}"," ", norm_text) #2개 이상 띄어쓰기 1개로 
        norm_text = re.sub( r"\A\s+",'', norm_text) #전처리로 생성된 맨 앞 띄어쓰기 제거
        result.append(norm_text)
    return result

norm_content = text_normalization(content)

#print(norm_content[:20])

In [14]:
### 명사 추출
noun_content = []
for x in norm_content:
    noun_content.append(mecab.nouns(x))

##불용어 제거
def delete_stopwords(norm_content, test=False):
    noun_content = []
    for x in norm_content:
        noun_content.append(mecab.nouns(x))
    
    ### 추출된 명사들을 묶어 merge_content 에 담는다
    merge_content = []
    for noun in noun_content:
        txt = ' '.join(noun)
        merge_content.append(txt)
    
    ### 불용어 리스트에 없는 단어들만 추가, 명사들로 이뤄진 리스트들
    texts = [[word for word in document.split() if word not in stop_words]
            for document in merge_content]
    if test == True:
        return texts[0]
    return texts

texts = delete_stopwords(norm_content)
#print(texts)


###texts 합쳐서 하나의 코퍼스로
# n_merge_content = []

# for noun in texts:
#     txt = ' '.join(noun)
#     n_merge_content.append(txt)




# 구축한 corpus로 doc2vec 모델 만들기

In [18]:
#corpus 읽어서 태그달기(태그는 임의의 태그, 입력된 순서대로 달림)
def read_corpus(noun_content):
    for i, content in tqdm(enumerate(noun_content)):
        yield gensim.models.doc2vec.TaggedDocument(content, [i])

corpus = list(read_corpus(noun_content))
print(noun_content[:3])
print("-"*100)
print(corpus[:3])
#모델 학습, doc2vec
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40) #모델 생성
model.build_vocab(corpus) #모델 훈련 위한 vocab 생성
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs) #모델 훈련

3024it [00:00, 571847.40it/s]
[['부', '전공', '신청', '학기', '가능', '막', '학기', '계절도', '예정', '졸', '건데', '이번', '학기', '중', '이중', '전공', '걸', '전공', '변경', '가능'], ['저', '전역', '주', '정도', '군인', '전역', '후', '계획', '점', '질문', '저', '군', '복무', '중', '군', '원격', '강좌', '년', '학기', '교양', '학점', '수강', '전역', '후', '년', '학기', '복학', '예정', '년', '여름', '계절', '학기', '것', '가능', '가요', '만약', '여름', '계절', '학기', '수강', '것', '가능', '년', '학기', '입대', '전', '학기', '수강', '강좌', '년', '여름', '계절', '학기', '것', '가능', '학우', '노력', '께', '감사', '말씀'], ['방학', '중', '어학연수', '질문', '휴학', '중', '학점', '인정', '재학', '기간', '중', '회', '가능', '데', '방학', '중', '어학연수', '휴학', '중', '학점', '인정', '기준', '방학', '휴학', '중', '어학연수', '관련', '내용', '휴학', '중', '학점', '인정', '회', '가능']]
----------------------------------------------------------------------------------------------------
[TaggedDocument(words=['부', '전공', '신청', '학기', '가능', '막', '학기', '계절도', '예정', '졸', '건데', '이번', '학기', '중', '이중', '전공', '걸', '전공', '변경', '가능'], tags=[0]), TaggedDocument(words=['저', '전역', '주', '정도', '군인', '전역'

# 유사한 상위 10개 질문들과 그 답변들

In [19]:
#main
query = input('궁금하신 사항을 입력해 주세요: ')
query = [query] #전처리에 인수 값으로 리스트 형식으로 넣어줘야 하기 때문에 문자열을 포함한 리스트 형식으로 변경

##전처리 진행, 전처리한 문장을 학습한 doc2vec모델에 적용해 vectorize
norm_content = text_normalization(query)
preprocessed = delete_stopwords(norm_content, test = True)
vector = model.infer_vector(preprocessed) #학습된 모델로 벡터값 유추

##가장 유사한 vector를 갖는 질문들 모아모아
sims = model.docvecs.most_similar([vector], topn=len(model.docvecs))

##유사도 상위 10개 tag들
sims_tags = list(map(lambda x: x[0], sims[:10])) 


print('\n\n')
for i, tag in enumerate(sims_tags):
    print(i+1, ')', '유사도: ', sims[i][1])
    print('------질문------')
    print(content[tag])
    print('\n')
    
    print('------답변------')
    comment_devided = comments[tag].split('[')[1].split(']')[0].split("\', \'")
    for j, comment in enumerate(comment_devided):
        print(j+1,')', comment)
    print('\n\n\n\n')




1 ) 유사도:  0.8827359080314636
------질문------
안녕하세요 조교님! 같은 전공 부전공에서 이중으로 올리는 것도 혹시 이중전공 변경 신청 해야하나요?


------답변------
1 ) '네 이중전공을 신청하는 것이므로 변경신청을 통해 해야합니다'





2 ) 유사도:  0.8716222047805786
------질문------
서울캠 > 글캠 통계학과이중 혹은 부전공 가능한가요?


------답변------
1 ) '네 이공계열(공대, 자대) 소속 전공들은 다 가능합니다~'





3 ) 유사도:  0.8669173717498779
------질문------
조교님! 혹시 부전공을 이중전공으로 올리려면 어떻게 해야되나요?ㅜㅜ


------답변------
1 ) '이중전공 변경기간에 동일하게 신청하셔야 됩니다 단 이전에 이중전공 변경 이력이 있으면 재학 중 1회라는 규정상 신청이 불가합니다'





4 ) 유사도:  0.8663200736045837
------질문------
안녕하세요 조교님 ㅠㅠ 이중전공을 부전공으로 내렸는데 학점 일부분이 자선으로 넘어갔어요ㅠㅠ 자선을 부전공으로 인정받으려면 어캐해야하나요ㅠ


------답변------
1 ) '소속 단과대에 이수구분변경 신청서를 내야합니다 단과대 사무실에 문의 바랍니다
2 ) 소속단과대는 부전공단과대 말씀이신가요? 아님 본전공?
3 ) 답변을 헷갈리게 했군요 부전공 단과대에 문의바랍니다
4 ) 감사합니다..'





5 ) 유사도:  0.8570381999015808
------질문------
안녕하세요 이번학기 휴학 신청을 하고 등록휴학을 하려고 햇는데 깜빡하고 등록금 납부를 못했습니다. 혹시 추가 등록기간에 등록금을 완납하면 등록휴핟 가능할까요?


------답변------
1 ) '추가로 국장도 그대로 받을수 있을까요..?
2 ) 휴학 신청을 한 상태에서 금번 등록기간에 납부를 했든 추가등록기간에 납부를 하든 다 등록휴학이 됩니다 국장에 선발되셨다